In [12]:
import os
import pandas as pd


In [14]:
df_recommender_path = os.path.join("exports/df_recommender.pkl")
df_recommender = pd.read_pickle(df_recommender_path)
df_recommender['subject_id'] = df_recommender['subject_id'].astype(str).str.strip()
df_recommender['node label'] = df_recommender['node label'].astype(str).str.strip()
df_recommender['director'] = df_recommender['director'].astype(str).str.strip()
df_recommender['performer'] = df_recommender['performer'].astype(str).str.strip()
df_recommender['genre'] = df_recommender['genre'].astype(str).str.strip()
df_recommender['screenwriter'] = df_recommender['screenwriter'].astype(str).str.strip()


In [15]:
print(df_recommender.head())
print(len(df_recommender))

predicate_label                               subject_id  \
0                http://www.wikidata.org/entity/Q1000825   
1                http://www.wikidata.org/entity/Q1000826   
2                http://www.wikidata.org/entity/Q1001759   
3                http://www.wikidata.org/entity/Q1001777   
4                http://www.wikidata.org/entity/Q1001943   

predicate_label                                        cast member  \
0                wd:Q16305292,pimprapa tangprabhaporn,santisuk ...   
1                wende wagner,fernando rey,frank silvera,michae...   
2                billy chow,shinobu nakayama,toshimichi takahas...   
3                silk smitha,poornam vishwanathan,sridevi,kamal...   
4                burt lancaster,pat mccormick,denver pyle,geral...   

predicate_label                  director performer  \
0                         nonzee nimibutr       nan   
1                            paul wendkos       nan   
2                gordon chan,yuen wooping       nan  